In [266]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
import numpy as np

In [93]:
incidents_fn = "data/20220413_D1_Incidents.csv"
rule_book_fn = "data/rule_book.csv"

In [94]:
rules = pd.read_csv(rule_book_fn)
incidents = pd.read_csv(incidents_fn, dtype=str)

In [95]:
incidents.head(3)

,tblID,IRN,IncidentNumber,IncidentDate,TimeofIncident,BusinessUnit,BusinessGroup,ServiceLine,Project,Office,...,JFCAction,OHdefinition,MTCcount,RWCcount,LTIcount,FAcount,TransferTimeStamp,SourceFileName,FullDescription,ImmediateAction
0,175,39878,20200472,26/02/2020,14:15,Projects,Renewable Energy & Power,Power & Industrials EPC,YCI M1 Project,YCI M1 Project,...,NaN,NaN,NaN,NaN,NaN,1,NaN,CAIRS_Incidents.csv,Employee was grinding with a four-inch grinder...,Employee was transported to on-site medical an...
1,176,39877,20200471,26/02/2020,16:15,Consulting,Energy Optimisation & Innovation,EMEA,E&II Sustainable Infrastructure,E&II Sustainable Infrastructure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIRS_Incidents.csv,See supporting information,"I stopped the drillers from installing, phoned..."
2,177,39876,20200470,27/02/2020,10:15,Operations,Asia Pacific,APAC East,Esso PNG LNG,Hides,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIRS_Incidents.csv,At approximately 10:15 on the 27th of Feb a hy...,"All contaminated materials, absorbent pads, we..."


In [96]:
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

incidents = incidents[['incident_id', 'text']]

In [315]:
incidents.text[0]

"foreign body entered employee's (l) eye while grinding. employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye. employee was wearing all required ppe. investigation is pending upon employee’s return from offsite medical evaluation. it will be noted that the site has been dealing with strong winds throughout the day. foreign body was removed using first aid measures. employee was transported to on-site medical and then transferred to an off-site eye specialist for further treatment. foreign body was removed using first aid measures. employee was released to return to work without restrictions."

In [326]:
import nltk
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

test = incidents.text.head(1).apply(lemmatize_text)
print(list(test))

[['foreign', 'body', 'entered', "employee's", '(l)', 'eye', 'while', 'grinding.', 'employee', 'wa', 'grinding', 'with', 'a', 'four-inch', 'grinder', 'on', 'an', 'iron', 'support', 'in', 'the', 'compressor', 'building', 'when', 'he', 'felt', 'discomfort', 'to', 'his', 'left', 'eye.', 'employee', 'wa', 'wearing', 'all', 'required', 'ppe.', 'investigation', 'is', 'pending', 'upon', 'employee’s', 'return', 'from', 'offsite', 'medical', 'evaluation.', 'it', 'will', 'be', 'noted', 'that', 'the', 'site', 'ha', 'been', 'dealing', 'with', 'strong', 'wind', 'throughout', 'the', 'day.', 'foreign', 'body', 'wa', 'removed', 'using', 'first', 'aid', 'measures.', 'employee', 'wa', 'transported', 'to', 'on-site', 'medical', 'and', 'then', 'transferred', 'to', 'an', 'off-site', 'eye', 'specialist', 'for', 'further', 'treatment.', 'foreign', 'body', 'wa', 'removed', 'using', 'first', 'aid', 'measures.', 'employee', 'wa', 'released', 'to', 'return', 'to', 'work', 'without', 'restrictions.']]


In [329]:
lem.lemmatize('employee’s')

'employee’s'

In [331]:
# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
stem = PorterStemmer()
def stem_text(text):
    return [stem.stem(w) for w in w_tokenizer.tokenize(text)]
   
test = incidents.text.head(1).apply(stem_text)
print(list(test))

program  :  program
programs  :  program
programmer  :  programm
programming  :  program
programmers  :  programm
[['foreign', 'bodi', 'enter', "employee'", '(l)', 'eye', 'while', 'grinding.', 'employe', 'wa', 'grind', 'with', 'a', 'four-inch', 'grinder', 'on', 'an', 'iron', 'support', 'in', 'the', 'compressor', 'build', 'when', 'he', 'felt', 'discomfort', 'to', 'hi', 'left', 'eye.', 'employe', 'wa', 'wear', 'all', 'requir', 'ppe.', 'investig', 'is', 'pend', 'upon', 'employee’', 'return', 'from', 'offsit', 'medic', 'evaluation.', 'it', 'will', 'be', 'note', 'that', 'the', 'site', 'ha', 'been', 'deal', 'with', 'strong', 'wind', 'throughout', 'the', 'day.', 'foreign', 'bodi', 'wa', 'remov', 'use', 'first', 'aid', 'measures.', 'employe', 'wa', 'transport', 'to', 'on-sit', 'medic', 'and', 'then', 'transfer', 'to', 'an', 'off-sit', 'eye', 'specialist', 'for', 'further', 'treatment.', 'foreign', 'bodi', 'wa', 'remov', 'use', 'first', 'aid', 'measures.', 'employe', 'wa', 'releas', 'to', 'retu

In [98]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [tok for tok in tokens if tok!='-']
    return tokens

In [99]:
test_tokens = tokenize(incidents.text[0])

In [100]:
def translate_to_regex(rule_part):
    """
    converts rule_book rules to regex format;
    drops trailing separators in process
    """
    if isinstance(rule_part, str):
        rule_part = re.sub(r"\s{2,}", " ", rule_part)
        rule_part = re.sub(r'\\\\b', r'\\b', rule_part)
        
        return r'(\b' + r'\b)|(\b'.join([s for s in rule_part.split('_ ') if s]) + r'\b)'
    else:
        return ''

In [101]:
rules['keyword'] = rules['keyword'].apply(lambda x: x.replace('*', '.*')+r'\b')
rules['rules_pre'] = rules['rules_pre'].apply(translate_to_regex)
rules['rules_post'] = rules['rules_post'].apply(translate_to_regex)
rules['rules_all'] = rules['rules_all'].apply(translate_to_regex)

rules['voids'] = rules['voids'].apply(translate_to_regex)
print('rules translated')

rules translated


In [102]:
rules

,group,keyword,rules_pre,rules_post,rules_all,voids
0,eye injury or irritation,eye.*\b,(\bsomething.*in\b)|(\bswell.*\b)|(\bswollen\b...,,(\birritation\b)|(\bforeign body\b)|(\bforeign...,
1,site compliance or practice issue,permit\b,(\bwithout.*work\b)|(\bnot captured\b),(\bnot in place\b)|(\bwere not captured\b),(\bduring.* audit\b)|(\bfailed.* section\b)|(\...,


In [105]:
def check_presence(pattern, string):
    if pattern:
        return bool(re.search(pattern, string))
    else:
        return False

In [109]:
def find_pattern(tokens, keyword, check_pre, check_post, check_all, check_void, window):
    """
    for a list of tokens finds specified keyword and returns True
    if the neighbourhood of this keyword satisfies pre-, post- or all- context rules
    and doesn't contain anything forbidden
    :param tokens: list of tokens
    :param keyword: pattern which a token should match
    :param check_pre: pattern which several previous tokens (concatenated) should match
    :param check_post: pattern which several subsequent tokens (concatenated) should match
    :param check_all: pattern which previous tokens + keyword + subsequent tokens should match
    :param check_void: pattern which previous tokens + keyword + subsequent tokens should NOT match
    :param window: N of pre and post tokens to consider
    :return: True/False - whether at least one matching part was found
    """
    # Extract contexts of keyword (if any found)
    # Four parts: [(pre), (keyword), (post), (all)]
    matches = [(
        ' '.join(tokens[max(0, i - window - 1): i]), tokens[i],
        ' '.join(tokens[i + 1:i + window + 1]),
        ' '.join(tokens[max(0, i - window - 1):i + window + 1])
        ) for i, tok in enumerate(tokens) if re.match(keyword, tok)]
    
    # do tests
    final_match = any([
        (check_presence(check_pre, pre) or check_presence(check_all, all_) or check_presence(check_post, post))
        and not check_presence(check_void, all_) for (pre, kw, post, all_) in matches
    ])
    return final_match

In [115]:
tokens = test_tokens
keyword = rules.keyword[0]
check_pre = rules.rules_pre[0]
check_post = rules.rules_post[0]
check_all = rules.rules_all[0]
check_void = rules.voids[0]
window = 5

In [116]:
matches = [(
        ' '.join(tokens[max(0, i - window - 1): i]), tokens[i],
        ' '.join(tokens[i + 1:i + window + 1]),
        ' '.join(tokens[max(0, i - window - 1):i + window + 1])
        ) for i, tok in enumerate(tokens) if re.match(keyword, tok)]

In [117]:
matches

[("entered employee 's ( l )",
  'eye',
  'while grinding . employee was',
  "entered employee 's ( l ) eye while grinding . employee was"),
 ('he felt discomfort to his left',
  'eye',
  '. employee was wearing all',
  'he felt discomfort to his left eye . employee was wearing all'),
 ('and then transferred to an off-site',
  'eye',
  'specialist for further treatment .',
  'and then transferred to an off-site eye specialist for further treatment .')]

In [111]:
find_pattern(
    tokens = test_tokens, 
    keyword = rules.keyword[0], 
    check_pre = rules.rules_pre[0], 
    check_post = rules.rules_post[0], 
    check_all = rules.rules_all[0], 
    check_void = rules.voids[0], 
    window = 5
)

True

In [119]:
[(check_presence(check_pre, pre) or check_presence(check_all, all_) or check_presence(check_post, post))
        and not check_presence(check_void, all_) for (pre, kw, post, all_) in matches]

[False, True, False]

In [125]:
for (pre, kw, post, all_) in matches:
    #print(pre)
    #print(kw)
    #print(post)
    print(all_)

entered employee 's ( l ) eye while grinding . employee was
he felt discomfort to his left eye . employee was wearing all
and then transferred to an off-site eye specialist for further treatment .


In [129]:
check_all

'(\\birritation\\b)|(\\bforeign body\\b)|(\\bforeign object\\b)|(\\birritation (.*) eye\\b)|(\\bpain in (.*) eye\\b)|(\\bfell (.*) eye\\b)|(\\bfall (.*) eye\\b)|(\\bdebris (.*) eye\\b)|(\\bpain to\\b)'

In [283]:
syn_dict = {
    'worker_syns':['employee', 'ip', 'worker', 'man', 'woman'],
    'pain_syns':['pain', 'discomfort', 'soreness', 'hurt', 'something in', 
             'swelling', 'swollen', 'dust', 'particle', 'irritation', 'foreign'],
    'eye_syns':['eye', 'pupil', 'iris']
}

In [206]:
chk_text = 'the employee felt discomfort in his eye'
pattern = '(\\bemployee (.*) discomfort in (.*) eye\\b)'
check_presence(pattern, chk_text)

True

In [261]:
chk_text = 'the ip felt extreme soreness in his eye'
finds_list = []
finds_pats = []
incid_nums = []
for worker_syn in worker_syns:
    for pain_syn in pain_syns: 
        for eye_syn in eye_syns:   
            pattern = f'(\\b{worker_syn} (.*) {pain_syn} (.*) {eye_syn}\\b)'
            check = check_presence(pattern, chk_text)
            if check: 
                print(f'{check}: {pattern}')
                finds_list.append(check)
                finds_pats.append(pattern)
                incid_nums.append(irn)

True: (\bip (.*) soreness (.*) eye\b)


In [284]:
syn_dict.get('worker_syns')

['employee', 'ip', 'worker', 'man', 'woman']

In [292]:
srs = ([2, 1, 4, 3, 0], [])

In [293]:
srs[1]

[]

In [314]:
pos_1st = syn_dict.get('worker_syns')
pos_2nd = syn_dict.get('pain_syns')
pos_3rd = syn_dict.get('eye_syns')

# Connections between synonyms 1-2, 2-3
connect = ['(.*)', '(.*)']

# Shuffle rules
srs = ([2, 1, 4, 3, 0], [])

finds_list = []
finds_pats = []
incid_nums = []

for row in tqdm(range(len(incidents))):
    chk_text = incidents.text[row].lower()
    lem_text = lem.lemmatize(chk_text)
    irn = incidents.incident_id[row]
    for first_syn in pos_1st:
        locals()["first_syn"] = first_syn
        for second_syn in pos_2nd: 
            for third_syn in pos_3rd:  
                x = f'{first_syn} {connect[0]} {second_syn} {connect[1]} {third_syn}'
                pattern = f'(\\b{x}\\b)'
                check = check_presence(pattern, chk_text)
                if check: 
                    print(f'{check}: {pattern}')
                    finds_list.append(check)
                    finds_pats.append(pattern)
                    incid_nums.append(irn)
                else:
                    for sr in srs:
                        if len(sr) == 0: continue
                        a = x.split()
                        # Adjust the word sequence using shuffle rule
                        pattern = f'(\\b{a[sr[0]]} {a[sr[1]]} {a[sr[2]]} {a[sr[3]]} {a[sr[4]]}\\b)'
                        rev_check = check_presence(pattern, chk_text)
                        if rev_check: 
                            print(f'{rev_check}: {pattern}')
                            finds_list.append(check)
                            finds_pats.append(pattern)
                            incid_nums.append(irn)
                    

  0%|          | 0/27158 [00:00<?, ?it/s]

foreign body entered employee's (l) eye while grinding. employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye. employee was wearing all required ppe. investigation is pending upon employee’s return from offsite medical evaluation. it will be noted that the site has been dealing with strong winds throughout the day. foreign body was removed using first aid measures. employee was transported to on-site medical and then transferred to an off-site eye specialist for further treatment. foreign body was removed using first aid measures. employee was released to return to work without restrictions.
True: (\bemployee (.*) discomfort (.*) eye\b)
True: (\bemployee (.*) foreign (.*) eye\b)
drainage pipe damaged at ~2.2 m depth. see supporting information i stopped the drillers from installing, phoned the site manager to report a probable service strike, who contacted the project manager. i then spoke to the project man

small cut in a finger while cutting a cable tray with a hacksaw an electrician got a cut in a finger while cutting a cable tray with a hacksaw. cut occurred when the hand got in contact with the sharp edge of the remaining cable tray. cut inhibiting gloves were used.  contacted the nurse and got the wound / cut cleansed and patched. changed to new gloves.
debris on eye when mounting grating (strong wind in the area) when mounting grating outside the glycol / diesel treatment room, the operator got a little debris on the eye. it was blowing over 40 knots at the time, so sand blasting grains was taken by the wind and caught the eye. safety glasses were used.comments: sand was left behind after a previous job. high pressure washers were out of order. it was swept before work started. went to the nurse got rinsed.
oil leak from crane that was on site for planned work. a line of oil was noted on the pebbled area at separation 1 the day after a crane had been in situ for a planned lift. from

ip recieved a leach bite to his leg drilling contractor reported a bite on his left leg at kp6.ip escorted to clinic where, wound area cleaned and dressed and the person was sent straight back to work. it is believed ip may have got the leech onto him via his rubber boots which he leaves at site each night. ip removed the leech immediately while onsite.   post incident action:work crew were addressed this morning about correct storage of ppe and inspecting it prior to wearing. 
two wood company vehices were stolen from the wood seminole, tx. yard. at sometime before 6 a.m., feb. 12th, two wood company vehicles were stolen from the wood seminole tx. yard. due to the fact that both vehicles were equipped with geotab devices, trucks were located in neighboring town of hobbs, nm 30 miles west of seminole, tx. sheriff's office was contacted, vehicles were located and five persons have been arrested in hobbs, nm as a result.
walking in tool container, employee tripped on door threshold, put 

 puncture wound to palm of right hand team member (tm) was vacuuming out a hole with a vacuum truck, and the vacuum became clogged. tm went to clear the blocked section of the hose. he reached down to clear the blockage and lost his balance. he went to catch himself with his right hand on the ground, as he placed his hand on the ground, he felt a pain. tm raised his hand to find a small board with two screws sticking to the palm of his hand. tm notified superintendent, they cleaned the wound with over the counter medication, and wrapped the hand in clean bandages. the superintendent and the tm called occucare and were told the injury is a first aid case only. occucare recommended a tetanus shot for tm.
ip was grinding a weld when the grinder kicked back making contact with left hand causing small abrasion.  ip was tasked with removing a 3” 90 from a spool piece. this was completed successfully and now the 90  needed the bevel cleaned up to be welded. the 90 was the secured in a vise to

...

a piece of cable tray approx. 4 m in length dropped 4 m to the deck while installing a 2” air hose migrated from legacy cairs incident #34561rats were installing a 2” air hose from the ensco to the bravo air receiver. while running the hose the intention was to utilise where possible any cable racks. as the air hose was placed onto the cable rack on the workshop east wall, the corroded supporting brackets failed allowing a section of approx. 4m weighing 15kg to fall to the deck below. there was a permit issued for the task and a tool box talk was carried out by the rope access team. barriers were in place and no other persons were in the vicinity. the drop zone was such that the cable tray could not fall or bounce outside of the barriers. the double door to the work shop is locked shut with a scaffold arrangement so no personnel could access the work site this way. there was very little, or no chance of any personnel being struck by the cable tray in this instance. utilising the drops 

bracket supporting kennedy grating on staircase gave way causing grating to drop 2 inches when walked on. migrated from legacy cairs incident #34529jm was accessing work face after having morning break and returning back to site.jm took the north alimak to 151 level and accessed roof area before going to west side fire escape.on entering the west side fire escape the kennedy floor grating gave way as jm stepped onto it.floor grating dropped 2-3 inches and rested on pipe below grating.this was reported to g ford (ramboll co-ordinator) and subsequently wgis personnel immediately following incident.on initial investigation it looks like bracket has came loose / snapped causing grating to drop.initial root cause discussions look at lack of routine maintenance regarding grating and retaining clips.lynemouth power station to investigate and relay findings to wgis site team. migrated from legacy cairs - see detailed description
during daily checks crane operator noticed crane drip tray requir

two wg subcontractors employees were installing the thermal insulation in the high pressure wash line. while one of them was cutting the metal plate, the sharp edge of the plate scratched his third finger of the right hand. migrated from legacy cairs incident #34483it was planned to install the thermal insulation of the high pressure wash line by a wg subcontractor. when one subcontractor employee was cutting the metal plate, the sharp edge of plate scratched his third finger of right hand. the ip did not noticed the incident and, during a management walk around, he was advised that his glove was pierced and with blood. he took off the glove and realized that he had a small injury in his third finger.the activity was stopped and the ip went to the platform hospital for examination. the doctor confirmed only a superficial injury, performed the first aid, and allowed the ip going back to his normal duties with no restrictions. migrated from legacy cairs - see detailed description
ip graz

while transferring from the platform to the boat josh's knee hit the top of the stem bulwarks causing a minor abrasion. migrated from legacy cairs incident #34461while transferring from the platform to the boat josh's knee hit the top of the stem bulwarks causing a minor abrasion. no treatment. migrated from legacy cairs - see detailed description
adt 740 (pp106) experienced overflow of diesel due to heat expansion, approx. 0.5l diesel leaked out from the breather cap and dripped on the ground. migrated from legacy cairs incident #34452wpa angore yard. adt 740 (pp106) experienced overflow of diesel due to heat expansion, approx. 0.5l diesel leaked out from the breather cap and dripped on the ground. the adt was refuelled at 6am this morning (approx. 40l). the adt was parked up in wpa yard and not working. · where: wpa yard · what: &lt;0.5lt of diesel to grade · when: 23th august – 15:00 hrs · how: fuel tank was over filled which facilitated expansion to occur · other: refuellers observ

KeyboardInterrupt: 

In [312]:
len(list(dict.fromkeys(incid_nums)))

543

In [304]:
import nltk as nltk
from nltk.stem import WordNetLemmatizer

In [306]:
lem = WordNetLemmatizer()
test = lem.lemmatize(incidents.text)

TypeError: unhashable type: 'Series'

In [311]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df = pd.DataFrame(['this was cheesy', 'she likes these books', 'wow this is great'], columns=['text'])
df['text_lemmatized'] = df.text.apply(lemmatize_text)

TypeError: sequence item 0: expected str instance, list found

In [308]:
df

,text,text_lemmatized
0,this was cheesy,"[this, wa, cheesy]"
1,she likes these books,"[she, like, these, book]"
2,wow this is great,"[wow, this, is, great]"
